## TV definition

The total variation $\text{TV}(f)$ of an image $f(x,y,z,t)$ is here defined using the $L_{2,1}$ norm as

\begin{equation}
\text{TV}(f) = \left\|  \nabla f  \right\|_{2,1} = \sum_{i,j,k,l} \sqrt{ (\partial_x f_l)_{i,j,k}^2 + (\partial_y f_l)_{i,j,k}^2  + (\partial_z f_l)_{i,j,k}^2 + \mu (\partial_t f_l)_{i,j,k}^2} \;,
\end{equation}

in which $\mu$ is a time-regularization factor, and $l$ is the time-index.

To compute the TV in practice, the image $f$ is discretized in space and time as a series of 3D matrices $\mathrm{f}=(\mathrm{f}_1, \dots, \mathrm{f}_{N_t})$.
Different choices of discretization can then be used to approximate the square of the spatial and time gradients.
For instance, the equations below correspond to an upwind discretization:

\begin{align}
(\partial_x \mathrm{f}_l)^2_{i,j,k} & = \left( (\mathrm{f}_l)_{i+1,j,k} - (\mathrm{f}_l)_{i,j,k} \right)^2 \\
(\partial_y \mathrm{f}_l)^2_{i,j,k} & = \left( (\mathrm{f}_l)_{i,j+1,k} - (\mathrm{f}_l)_{i,j,k} \right)^2 \\
(\partial_z \mathrm{f}_l)^2_{i,j,k} & = \left( (\mathrm{f}_l)_{i,j,k+1} - (\mathrm{f}_l)_{i,j,k} \right)^2 \\
(\partial_t \mathrm{f}_l)^2_{i,j,k} & = \left( (\mathrm{f}_{l+1})_{i,j,k} - (\mathrm{f}_l)_{i,j,k} \right)^2
\end{align}

Although this scheme is commonly used in TV calculations, it only provides first-order Taylor approximation of the estimated gradients, and can result in a certain alignment bias.
In PyTV, four different discretizations are implemented, including a Hybrid Discretization Total Variation (HDTV) scheme that is both second order and mitigates the mentioned alignment bias.
The gradient discretization used in the four different schemes implemented are illustrated below for the case of $(\partial_x \mathrm{f}_l)^2_{i,j,k}$ in 2D: 



## General definitions

- 4D Image $f_{i,j,k,l}$ with $0 \leq (i,j) \leq N-1$, $0 \leq k \leq N_z-1$, and $0 \leq l \leq M-1$

- Row difference: $r_{i,j,k,l} = f_{i+1,j,k,l}-f_{i,j,k,l}$

- Column difference: $c_{i,j,k,l} = f_{i,j+1,k,l}-f_{i,j,k,l}$

- Slice difference: $s_{i,j,k,l} = f_{i,j,k+1,l}-f_{i,j,k,l}$

- Time difference: $t_{i,j,k,l} = f_{i,j,k,l+1}-f_{i,j,k,l}$

- Each direction is extended by 1 on both ends such that $r_{i,j,k,l}$, $c_{i,j,k,l}$, $s_{i,j,k,l}$, $t_{i,j,k,l}$ is defined for all $(i,j,k,l)$. We pick: $r_{N-1,j,k,l} = 0$ and $r_{-1,j,k,l} = 0$ for all $(j,k,l)$. Same convention for other 3 differences.

- 2D Example matrix:

$
A = \begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$

## TV Upwind


- $TVU(f) = \sum_{l = 0}^{M-1} \sum_{k = 0}^{N_z-1} \sum_{i,j = 0}^{N-1} G^U_{i,j,k,l}$

- $G^U_{i,j,k,l} = \sqrt{r_{i,j,k,l}^2+c_{i,j,k,l}^2+s_{i,j,k,l}^2+t_{i,j,k,l}^2}$

- Not differentiable everywhere, isotropic.

- $[\partial TVU(f)]_{m,n,p,q} = \partial_{m,n,p,q}(G^U_{m,n,p,q}) + \partial_{m,n,p,q}(G^U_{m-1,n,p,q}) \\ + \partial_{m,n,p,q}(G^U_{m,n-1,p,q}) + \partial_{m,n,p,q}(G^U_{m,n,p-1,q}) + \partial_{m,n,p,q}(G^U_{m,n,p,q-1}) \\= - \frac{r_{m,n,p,q}+c_{m,n,p,q}+s_{m,n,p,q}+t_{m,n,p,q}}{G^U_{m,n,p,q}}+\frac{r_{m-1,n,p,q}}{G^U_{m-1,n,p,q}}+\frac{c_{m,n-1,p,q}}{G^U_{m,n-1,p,q}}+\frac{s_{m,n,p-1,q}}{G^U_{m,n,p-1,q}} + \frac{t_{m,n,p,q-1}}{G^U_{m,n,p,q-1}}$

- Example: $TVU(A) = 2+\sqrt{2}$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 \\
0 & -1 & 2+\sqrt{2} & -1/\sqrt{2} & 0 \\
0 & 0 & -1/\sqrt{2} & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$


## TV Downwind


- $TVD(f) = \sum_{l = 0}^{M-1} \sum_{k = 0}^{N_z-1} \sum_{i,j = 0}^{N-1} G^D_{i,j,k,l}$

- $G^D_{i+1,j+1,k+1,l+1} = \sqrt{r_{i,j+1,k+1,l+1}^2+c_{i+1,j,k+1,l+1}^2+s_{i+1,j+1,k,l+1}^2+t_{i+1,j+1,k+1,l}^2}$

- Not differentiable everywhere, isotropic.

- $[\partial TVD(f)]_{m,n,p,q} = \partial_{m,n,p,q}(G^D_{m-1,n-1,p-1,q-1}) + \partial_{m,n,p,q}(G^D_{m,n-1,p-1,q-1}) \\ + \partial_{m,n,p,q}(G^D_{m-1,n,p-1,q-1}) + \partial_{m,n,p,q}(G^D_{m-1,n-1,p,q-1}) + \partial_{m,n,p,q}(G^D_{m-1,n-1,p-1,q}) \\= \frac{r_{m-1,n,p,q}+c_{m,n-1,p,q}+s_{m,n,p-1,q}+t_{m,n,p,q-1}}{G^D_{m-1,n-1,p-1,q-1}}-\frac{r_{m,n,p,q}}{G^D_{m,n-1,p-1,q-1}}-\frac{c_{m,n,p,q}}{G^D_{m-1,n,p-1,q-1}}-\frac{s_{m,n,p,q}}{G^D_{m-1,n-1,p,q-1}}-\frac{t_{m,n,p,q }}{G^D_{m-1,n-1,p-1,q}}$

- Example: $TVD(A) = 2+\sqrt{2}$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 \\
0 & -1 & 2+\sqrt{2} & -1/\sqrt{2} & 0 \\
0 & 0 & -1/\sqrt{2} & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$

## TV Downwind v2


- $TVD(f) = \sum_{l = 0}^{M-1} \sum_{k = 0}^{N_z-1} \sum_{i,j = 0}^{N-1} G^D_{i,j,k,l}$

- $G^D_{i,j,k,l} = \sqrt{r_{i-1,j,k,l}^2+c_{i,j-1,k,l}^2+s_{i,j,k-1,l}^2+t_{i,j,k,l-1}^2}$

- Not differentiable everywhere, isotropic.

- $[\partial TVD(f)]_{m,n,p,q} = \partial_{m,n,p,q}(G^D_{m,n,p,q}) + \partial_{m,n,p,q}(G^D_{m+1,n,p,q}) \\ + \partial_{m,n,p,q}(G^D_{m,n+1,p,q}) + \partial_{m,n,p,q}(G^D_{m,n,p+1,q}) + \partial_{m,n,p,q}(G^D_{m,n,p,q+1}) \\= \frac{r_{m-1,n,p,q}+c_{m,n-1,p,q}+s_{m,n,p-1,q}+t_{m,n,p,q-1}}{G^D_{m,n,p,q}}-\frac{r_{m,n,p,q}}{G^D_{m+1,n,p,q}}-\frac{c_{m,n,p,q}}{G^D_{m,n+1,p,q}}-\frac{s_{m,n,p,q}}{G^D_{m,n,p+1,q}}-\frac{t_{m,n,p,q }}{G^D_{m,n,p,q+1}}$

- Example: $TVD(A) = 2+\sqrt{2}$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 \\
0 & -1 & 2+\sqrt{2} & -1/\sqrt{2} & 0 \\
0 & 0 & -1/\sqrt{2} & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$


## TV Downwind old


- $TVD(f) = \sum_{i,j = 1}^{N-1} \sqrt{r_{i-1,j}^2+c_{i,j-1}^2} = \sum_{i,j = 0}^{N-2} \sqrt{r_{i,j+1}^2+c_{i+1,j}^2} =  \sum_{i,j = 0}^{N-2} G^D_{i,j}$

- Not differentiable everywhere, isotropic.

- $G^D_{i,j} = \sqrt{r_{i,j+1}^2+c_{i+1,j}^2}$
- $G^D_{i,j,k,l} = \sqrt{r_{i,j+1,k+1,l}^2+c_{i+1,j,k+1,l}^2+s_{i+1,j+1,k,l}^2+t_{i+1,j+1,k+1,l-1}^2}$

- $[\partial TVD(f)]_{p,q} = \partial_{p,q}(G^D_{p-1,q-1}) + \partial_{p,q}(G^D_{p-1,q}) + \partial_{p,q}(G^D_{p,q-1}) = \frac{r_{p-1,q}+c_{p,q-1}}{G^D_{p-1,q-1}} - \frac{c_{p,q}}{G^D_{p-1,q}} - \frac{r_{p,q}}{G^D_{p,q-1}}$

- $TVD(A) = 2+\sqrt{2}$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1/\sqrt{2} & 0 & 0 \\
0 & -1/\sqrt{2} & 2+\sqrt{2} & -1 & 0 \\
0 & 0 & -1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$


## TV Mixed

- $TVM(f) = \frac{1}{\sqrt{2}}\sum_{i,j = 0}^{N-2} \sqrt{r_{i,j}^2 + r_{i,j+1}^2 + c_{i,j}^2 + +c_{i+1,j}^2} = \frac{1}{\sqrt{2}} \sum_{i,j = 0}^{N-2} G^H_{i,j}$

- Not differentiable everywhere, isotropic.

- $G^H_{i,j} = \sqrt{r_{i,j}^2 + c_{i,j}^2 + r_{i,j+1}^2+c_{i+1,j}^2}$

- $[\partial TVM(f)]_{p,q} = \partial_{p,q}(G^H_{p,q}) + \partial_{p,q}(G^H_{p,q-1})+ \partial_{p,q}(G^H_{p-1,q})  + \partial_{p,q}(G^H_{p-1,q-1})  = -\frac{r_{p,q}+c_{p,q}}{G^H_{p,q}} + \frac{c_{p,q-1}-r_{p,q}}{G^H_{p,q-1}} + \frac{r_{p-1,q}-c_{p,q}}{G^H_{p-1,q}} + \frac{r_{p-1,q}+c_{p,q-1}}{G^H_{p-1,q-1}}$

- $TVM(A) = 4.0$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -2 & 0 & 0 \\
0 & -2 & 8 & -2 & 0 \\
0 & 0 & -2 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$


## TV Central

- $TVC(f) = \frac{1}{2} \sum_{i,j = 0}^{N-3} \sqrt{(r_{i+1,j}+r_{i,j})^2 + (c_{i,j+1} +c_{i,j})^2} = \frac{1}{2} \sum_{i,j = 0}^{N-3} G^C_{i,j}$

- Not differentiable everywhere, isotropic.

- $G^C_{i,j} = \sqrt{(r_{i+1,j}+r_{i,j})^2 + (c_{i,j+1} +c_{i,j})^2}$

- $[\partial TVC(f)]_{p,q} = \partial_{p,q}(G^C_{p,q}) + \partial_{p,q}(G^C_{p,q-1})+ \partial_{p,q}(G^C_{p-1,q})  + \partial_{p,q}(G^C_{p,q-2})  + \partial_{p,q}(G^C_{p-2,q})  = -\frac{r_{p+1,q}+r_{p,q}+c_{p,q+1}+c_{p,q}}{G^C_{p,q}} - 0 - 0+ \frac{r_{p-1,q}+r_{p-2,q}}{G^C_{p-2,q}} + \frac{c_{p,q-1}+c_{p,q-2}}{G^C_{p,q-2}}$

- $TVM(A) = 2.0$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & -1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
-1 & 0 & 4 & 0 & -1 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 \\
\end{pmatrix}
$


## TV Sum


- $TVS(f) = (TVD(f)+TVU(f))/2$

- Not differentiable everywhere, isotropic.

- Can't be written from the standard TV definition using a numerical discretization of $\left( \frac{\partial f}{\partial x}\right) ^2$.

- $TVS(A) = 2+\sqrt{2}$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -\frac{1+\sqrt{2}}{2\sqrt{2}} & 0 & 0 \\
0 & -\frac{1+\sqrt{2}}{2\sqrt{2}} & 2+\sqrt{2} & -\frac{1+\sqrt{2}}{2\sqrt{2}} & 0 \\
0 & 0 & -\frac{1+\sqrt{2}}{2\sqrt{2}} & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$

